In [1]:
#https://github.com/lukemelas/EfficientNet-PyTorch
!pip install efficientnet_pytorch

In [3]:
from fastai.script import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.distributed import *
from fastprogress import fastprogress
from torchvision.models import *
from efficientnet_pytorch import EfficientNet
import sys

torch.backends.cudnn.benchmark = True
fastprogress.MAX_COLS = 80

def get_data(size, woof, bs, workers=None):
    if   size<=128: path = URLs.IMAGEWOOF_160 if woof else URLs.IMAGENETTE_160
    elif size<=224: path = URLs.IMAGEWOOF_320 if woof else URLs.IMAGENETTE_320
    else          : path = URLs.IMAGEWOOF     if woof else URLs.IMAGENETTE
    path = untar_data(path)

    n_gpus = num_distrib() or 1
    if workers is None: workers = min(8, num_cpus()//n_gpus)

    return (ImageList.from_folder(path).split_by_folder(valid='val')
            .label_from_folder().transform(([flip_lr(p=0.5)], []), size=size)
            .databunch(bs=bs, num_workers=workers))

In [11]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_name('efficientnet-b0')

In [12]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b0')

Downloading: "http://storage.googleapis.com/public-models/efficientnet-b0-08094119.pth" to /home/dc/.torch/models/efficientnet-b0-08094119.pth
100%|██████████| 21383481/21383481 [00:02<00:00, 7874370.48it/s]


Loaded pretrained weights for efficientnet-b0


In [17]:
# Preprocess image
import json
from PIL import Image
import torch
from torchvision import transforms

tfms = transforms.Compose([transforms.Resize(224), transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])
img = tfms(Image.open('img.jpg')).unsqueeze(0)
print(img.shape) # torch.Size([1, 3, 224, 224])

# Load ImageNet class names
labels_map = json.load(open('labels_map.txt'))
labels_map = [labels_map[str(i)] for i in range(1000)]

# Classify
model.eval()
with torch.no_grad():
    outputs = model(img)

# Print predictions
print('-----')
print(type(torch.topk(outputs, k=5)))
print(torch.topk(outputs, k=5))
for idx in torch.topk(outputs, k=5).indices.squeeze(0).tolist():
    prob = torch.softmax(outputs, dim=1)[0, idx].item()
    print('{label:<75} ({p:.2f}%)'.format(label=labels_map[idx], p=prob*100))

torch.Size([1, 3, 224, 254])
-----
<class 'tuple'>
(tensor([[9.4877, 4.5943, 3.5910, 2.9450, 2.9364]]), tensor([[388, 296, 387, 270, 279]]))


AttributeError: 'tuple' object has no attribute 'indices'

In [4]:
data = get_data(300,1,16)   #240, bs=32 for B1, 300, bs=16 for B3

In [5]:
opt_func = partial(optim.Adam, betas=(0.9,0.99), eps=1e-6)

In [6]:
m = EfficientNet.from_name('efficientnet-b3')

In [7]:
m._fc = nn.Linear(m._fc.in_features, out_features=10, bias=True)
nn.init.kaiming_normal_(m._fc.weight);

In [8]:
learn = (Learner(data, m, wd=1e-5, opt_func=opt_func,
         metrics=[accuracy,top_k_accuracy],
         bn_wd=False, true_wd=True,
         loss_func = LabelSmoothingCrossEntropy())
        )

In [9]:
mixup = 0
if mixup: learn = learn.mixup(alpha=mixup)
learn = learn.to_fp16(dynamic=True)

In [10]:
#B1
learn.lr_find()
learn.recorder.plot()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


RuntimeError: _th_fill_ is not implemented for type torch.HalfTensor

In [ ]:
learn.fit_one_cycle(5, 1e-4, div_factor=10, pct_start=0.3)

In [ ]:
# restarted from scratch
learn.fit_one_cycle(5, 1e-3, div_factor=10, pct_start=0.3)

In [ ]:
# restarted from scratch
learn.fit_one_cycle(5, 5e-3, div_factor=10, pct_start=0.3)

In [ ]:
# restarted from scratch, mixup =0.2
learn.fit_one_cycle(80, 1e-3, div_factor=10, pct_start=0.3)

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, 1e-4, div_factor=10, pct_start=0.3)

In [ ]:
restart
learn.fit_one_cycle(5, 1e-3, div_factor=10, pct_start=0.3)

In [ ]:
#restart
learn.fit_one_cycle(5, 5e-5, div_factor=10, pct_start=0.3)